In [6]:
# Replicate local resources
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

In [7]:
# Download the .zip file with the content of the Kaggle contest
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

udea-ai4eng-20242.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
# Unzip the file with the content of the Kaggle contest
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
# List the unzipped files
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


In [10]:
# Import the needed libraries in this Colab
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# Read a load in dataframes the data of this project
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")
data_submission_example = pd.read_csv("submission_example.csv")

In [12]:
# Work on dataframe copies
data_train_copy = data_train.copy()
data_test_copy = data_test.copy()
data_submission_example_copy = data_submission_example.copy()

In [13]:
# Data view of first rows
data_train_copy.head(10)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,Más de 7 millones,Menos de 10 horas,Estrato 5,Si,Educación profesional completa,Secundaria (Bachillerato) completa,No,medio-alto
6,47159,20183,INGENIERIA MECANICA,HUILA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Educación profesional incompleta,Técnica o tecnológica completa,Si,alto
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria incompleta,Secundaria (Bachillerato) incompleta,Si,medio-bajo
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 1,Si,Secundaria (Bachillerato) completa,Educación profesional incompleta,Si,medio-bajo
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 5,Si,Postgrado,Postgrado,Si,alto


In [14]:
# Set the ID column as the index column
data_train_copy = data_train.set_index("ID")
data_train_copy.head(10)

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
ID,,,,,,,,,,,
904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo
659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,Más de 7 millones,Menos de 10 horas,Estrato 5,Si,Educación profesional completa,Secundaria (Bachillerato) completa,No,medio-alto
47159,20183,INGENIERIA MECANICA,HUILA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Educación profesional incompleta,Técnica o tecnológica completa,Si,alto
11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria incompleta,Secundaria (Bachillerato) incompleta,Si,medio-bajo
257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 1,Si,Secundaria (Bachillerato) completa,Educación profesional incompleta,Si,medio-bajo


# In the next cells I check the unique values of columns and check if they have NaN values.

In [15]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
data_train_copy['PERIODO'].unique(), data_train_copy['PERIODO'].unique().shape, data_train_copy['PERIODO'].isnull().any()

(array([20212, 20203, 20195, 20183, 20194, 20213, 20184, 20202, 20196]),
 (9,),
 False)

In [16]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
# In the future standardize the name of programs
data_train_copy['ESTU_PRGM_ACADEMICO'].unique(), data_train_copy['ESTU_PRGM_ACADEMICO'].unique().shape, data_train_copy['ESTU_PRGM_ACADEMICO'].isnull().any()

(array(['ENFERMERIA', 'DERECHO', 'MERCADEO Y PUBLICIDAD',
        'ADMINISTRACION DE EMPRESAS', 'PSICOLOGIA', 'MEDICINA VETERINARIA',
        'INGENIERIA MECANICA', 'ADMINISTRACIÓN EN SALUD OCUPACIONAL',
        'INGENIERIA INDUSTRIAL', 'ADMINISTRACIÓN FINANCIERA',
        'HOTELERIA Y TURISMO', 'LICENCIATURA EN CIENCIAS SOCIALES',
        'LICENCIATURA EN PEDAGOGIA INFANTIL', 'COMUNICACION SOCIAL',
        'CIENCIA POLITICA',
        'PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL',
        'MAESTRO EN MÚSICA', 'INGENIERIA MECATRONICA', 'TRABAJO SOCIAL',
        'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL',
        'INGENIERIA CIVIL', 'CONTADURIA PÚBLICA',
        'ADMINISTRACION EN SALUD', 'ADMINISTRACIÓN DE EMPRESAS',
        'ESTADISTICA', 'LICENCIATURA EN BIOLOGIA',
        'INGENIERIA AGROINDUSTRIAL', 'ZOOTECNIA',
        'COMUNICACION AUDIOVISUAL',
        'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES-INGLES',
        'COMUNICACION SOCIAL  - PERIODI

In [17]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
data_train_copy['ESTU_PRGM_DEPARTAMENTO'].unique(), data_train_copy['ESTU_PRGM_DEPARTAMENTO'].unique().shape, data_train_copy['ESTU_PRGM_DEPARTAMENTO'].isnull().any()

(array(['BOGOTÁ', 'ATLANTICO', 'SANTANDER', 'ANTIOQUIA', 'HUILA', 'SUCRE',
        'CAQUETA', 'CUNDINAMARCA', 'BOLIVAR', 'TOLIMA', 'VALLE', 'QUINDIO',
        'RISARALDA', 'CORDOBA', 'META', 'LA GUAJIRA', 'BOYACA', 'NARIÑO',
        'CAUCA', 'NORTE SANTANDER', 'CESAR', 'PUTUMAYO', 'CALDAS',
        'MAGDALENA', 'CHOCO', 'CASANARE', 'ARAUCA', 'GUAVIARE', 'AMAZONAS',
        'VAUPES', 'SAN ANDRES'], dtype=object),
 (31,),
 False)

In [18]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
data_train_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].unique(), data_train_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].unique().shape, data_train_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].isnull().any()

(array(['Entre 5.5 millones y menos de 7 millones',
        'Entre 2.5 millones y menos de 4 millones',
        'Entre 4 millones y menos de 5.5 millones', 'Más de 7 millones',
        'Entre 1 millón y menos de 2.5 millones',
        'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
        'No pagó matrícula', nan], dtype=object),
 (9,),
 True)

In [19]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
data_train_copy['ESTU_HORASSEMANATRABAJA'].unique(), data_train_copy['ESTU_HORASSEMANATRABAJA'].unique().shape, data_train_copy['ESTU_HORASSEMANATRABAJA'].isnull().any()

(array(['Menos de 10 horas', '0', 'Más de 30 horas', 'Entre 21 y 30 horas',
        'Entre 11 y 20 horas', nan], dtype=object),
 (6,),
 True)

In [20]:
# List of diferent values in the column
data_train_copy['FAMI_ESTRATOVIVIENDA'].unique(), data_train_copy['FAMI_ESTRATOVIVIENDA'].unique().shape, data_train_copy['FAMI_ESTRATOVIVIENDA'].isnull().any()

(array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
        nan, 'Estrato 6', 'Sin Estrato'], dtype=object),
 (8,),
 True)

In [21]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
data_train_copy['FAMI_TIENEINTERNET'].unique(), data_train_copy['FAMI_TIENEINTERNET'].unique().shape, data_train_copy['FAMI_TIENEINTERNET'].isnull().any()

(array(['Si', 'No', nan], dtype=object), (3,), True)

In [22]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
data_train_copy['FAMI_EDUCACIONPADRE'].unique(), data_train_copy['FAMI_EDUCACIONPADRE'].unique().shape, data_train_copy['FAMI_EDUCACIONPADRE'].isnull().any()

(array(['Técnica o tecnológica incompleta',
        'Técnica o tecnológica completa',
        'Secundaria (Bachillerato) completa', 'No sabe',
        'Primaria completa', 'Educación profesional completa',
        'Educación profesional incompleta', 'Primaria incompleta',
        'Postgrado', nan, 'Secundaria (Bachillerato) incompleta',
        'Ninguno', 'No Aplica'], dtype=object),
 (13,),
 True)

In [23]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
data_train_copy['FAMI_EDUCACIONMADRE'].unique(), data_train_copy['FAMI_EDUCACIONMADRE'].unique().shape, data_train_copy['FAMI_EDUCACIONMADRE'].isnull().any()

(array(['Postgrado', 'Técnica o tecnológica incompleta',
        'Secundaria (Bachillerato) completa', 'Primaria completa',
        'Técnica o tecnológica completa',
        'Secundaria (Bachillerato) incompleta',
        'Educación profesional incompleta',
        'Educación profesional completa', 'Primaria incompleta', nan,
        'Ninguno', 'No Aplica', 'No sabe'], dtype=object),
 (13,),
 True)

In [24]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
data_train_copy['ESTU_PAGOMATRICULAPROPIO'].unique(), data_train_copy['ESTU_PAGOMATRICULAPROPIO'].unique().shape, data_train_copy['ESTU_PAGOMATRICULAPROPIO'].isnull().any()

(array(['No', 'Si', nan], dtype=object), (3,), True)

In [25]:
# List of diferent values in the column
# Uniques, shape, does it have NaN values?
data_train_copy['RENDIMIENTO_GLOBAL'].unique(), data_train_copy['RENDIMIENTO_GLOBAL'].unique().shape, data_train_copy['RENDIMIENTO_GLOBAL'].isnull().any()

(array(['medio-alto', 'bajo', 'alto', 'medio-bajo'], dtype=object),
 (4,),
 False)

# Next, I start to clean up columns one by one and fill NaN values in a custom way from parents information beacause I think the parental backgroud is fuldamental in the destiny of any person.

In [26]:
# Replace 'Ninguno', 'No Aplica', and NaN values with 'No sabe'
data_train_copy['FAMI_EDUCACIONPADRE'] = data_train_copy['FAMI_EDUCACIONPADRE'].replace({
    'Ninguno': 'No sabe',
    'No Aplica': 'No sabe',
    np.nan: 'No sabe'
})

In [27]:
# Replace 'Ninguno', 'No Aplica', and NaN values with 'No sabe'
data_train_copy['FAMI_EDUCACIONMADRE'] = data_train_copy['FAMI_EDUCACIONMADRE'].replace({
    'Ninguno': 'No sabe',
    'No Aplica': 'No sabe',
    np.nan: 'No sabe'
})

In [28]:
# Replace NaN values with 'Sin Estrato'
data_train_copy['FAMI_ESTRATOVIVIENDA'] = data_train_copy['FAMI_ESTRATOVIVIENDA'].replace({
    np.nan: 'Sin Estrato'
})

In [29]:
# Replacement mappings based on education level
group1 = {'Postgrado', 'Educación profesional completa', 'Educación profesional incompleta'}
group2 = {'Técnica o tecnológica completa', 'Técnica o tecnológica incompleta', 'Secundaria (Bachillerato) completa', 'Secundaria (Bachillerato) incompleta'}
group3 = {'Primaria completa', 'Primaria incompleta'}

# List of all possible Estratos for random selection
all_estratos = ['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1', 'Estrato 6']
estrato_primaria = ['Estrato 1', 'Estrato 2']

# Custom function to perform replacements
def replace_sin_estrato(row):
    if row['FAMI_ESTRATOVIVIENDA'] != 'Sin Estrato':
        return row['FAMI_ESTRATOVIVIENDA']

    padre_edu = row['FAMI_EDUCACIONPADRE']
    madre_edu = row['FAMI_EDUCACIONMADRE']

    # Replace with 'Estrato 4' if any parent has a value in group1
    if padre_edu in group1 or madre_edu in group1:
        return 'Estrato 4'

    # Replace with 'Estrato 3' if any parent has a value in group2
    if padre_edu in group2 or madre_edu in group2:
        return 'Estrato 3'

    # Replace with 'Estrato 1' or 'Estrato 2' randomly if any parent has a value in group3
    if padre_edu in group3 or madre_edu in group3:
        return np.random.choice(estrato_primaria)

    # If both parents have 'No sabe', choose randomly from all estratos
    if padre_edu == 'No sabe' and madre_edu == 'No sabe':
        return np.random.choice(all_estratos)

    # Return the original value if none of the above rules match
    return row['FAMI_ESTRATOVIVIENDA']

# Apply the custom function to each row
data_train_copy['FAMI_ESTRATOVIVIENDA'] = data_train_copy.apply(replace_sin_estrato, axis=1)

# Display the DataFrame to verify the changes
data_train_copy['FAMI_ESTRATOVIVIENDA'].value_counts()


,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,237846
Estrato 3,220594
Estrato 1,117071
Estrato 4,73373
Estrato 5,27230
Estrato 6,16386


In [30]:
# Original DataFrame column
data_train['FAMI_ESTRATOVIVIENDA'].value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [31]:
# Replacement mappings based on average enrollment price
# Define the value mapping for `ESTU_VALORMATRICULAUNIVERSIDAD`
value_mapping = {
    'Más de 7 millones': 7.75,
    'Entre 5.5 millones y menos de 7 millones': 6.25,
    'Entre 4 millones y menos de 5.5 millones': 4.75,
    'Entre 2.5 millones y menos de 4 millones': 3.25,
    'Entre 1 millón y menos de 2.5 millones': 1.75,
    'Entre 500 mil y menos de 1 millón': 0.75,
    'Menos de 500 mil': 0.25,
    'No pagó matrícula': 0.00
}

# Replace the values in the `ESTU_VALORMATRICULAUNIVERSIDAD` column using the defined mapping
data_train_copy['ESTU_VALORMATRICULAUNIVERSIDAD'] = data_train_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(value_mapping)

# Create a custom function to handle `NaN` values based on `FAMI_ESTRATOVIVIENDA`
def replace_nan(row):
    if pd.notnull(row['ESTU_VALORMATRICULAUNIVERSIDAD']):
        return row['ESTU_VALORMATRICULAUNIVERSIDAD']

    estrato = row['FAMI_ESTRATOVIVIENDA']

    # Replace NaN based on `FAMI_ESTRATOVIVIENDA` column
    if estrato == 'Estrato 6':
        return 7.75
    elif estrato == 'Estrato 5':
        return 6.25
    elif estrato == 'Estrato 4':
        return 4.75
    elif estrato == 'Estrato 3':
        return np.random.choice([3.25, 1.75])
    elif estrato == 'Estrato 2':
        return np.random.choice([0.00, 0.25, 0.75])
    elif estrato == 'Estrato 1':
        return 0.00
    else:
        return row['ESTU_VALORMATRICULAUNIVERSIDAD']

# Apply the custom function to handle NaN values in the DataFrame
data_train_copy['ESTU_VALORMATRICULAUNIVERSIDAD'] = data_train_copy.apply(replace_nan, axis=1)

# Display the updated column values to verify changes
data_train_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()


<ipython-input-31-dc3cf3c9ac9c>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_train_copy['ESTU_VALORMATRICULAUNIVERSIDAD'] = data_train_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(value_mapping)


,count
ESTU_VALORMATRICULAUNIVERSIDAD,
1.75,204969
3.25,128330
0.25,80735
0.75,79175
4.75,70863
7.75,68607
6.25,39115
0.00,20706


In [32]:
# Original DataFrame column
data_train['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

,count
ESTU_VALORMATRICULAUNIVERSIDAD,
Entre 1 millón y menos de 2.5 millones,204048
Entre 2.5 millones y menos de 4 millones,127430
Menos de 500 mil,80263
Entre 500 mil y menos de 1 millón,78704
Entre 4 millones y menos de 5.5 millones,69736
Más de 7 millones,68014
Entre 5.5 millones y menos de 7 millones,38490
No pagó matrícula,19528


In [33]:
# Replacement mappings based on average working hours
# Define the value mapping
value_mapping = {
    'Más de 30 horas': 35,
    'Entre 21 y 30 horas': 25,
    'Entre 11 y 20 horas': 15,
    'Menos de 10 horas': 5,
    '0': 0
}

# Replace the values in the `ESTU_HORASSEMANATRABAJA` column using the defined mapping
data_train_copy['ESTU_HORASSEMANATRABAJA'] = data_train_copy['ESTU_HORASSEMANATRABAJA'].replace(value_mapping)

# Create a custom function to handle NaN values
def replace_nan_with_random(row):
    if pd.notnull(row['ESTU_HORASSEMANATRABAJA']):
        return row['ESTU_HORASSEMANATRABAJA']
    else:
        return np.random.choice([35, 25, 15, 5, 0])

# Apply the custom function to replace NaN values in the DataFrame
data_train_copy['ESTU_HORASSEMANATRABAJA'] = data_train_copy.apply(replace_nan_with_random, axis=1)

# Display the updated column values to verify changes
data_train_copy['ESTU_HORASSEMANATRABAJA'].value_counts()


<ipython-input-33-99b3eb9a3b31>:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_train_copy['ESTU_HORASSEMANATRABAJA'] = data_train_copy['ESTU_HORASSEMANATRABAJA'].replace(value_mapping)


,count
ESTU_HORASSEMANATRABAJA,
35.0,255621
0.0,122690
15.0,122017
25.0,98765
5.0,93407


In [34]:
data_train['ESTU_HORASSEMANATRABAJA'].value_counts()

,count
ESTU_HORASSEMANATRABAJA,
Más de 30 horas,249352
0,116550
Entre 11 y 20 horas,115857
Entre 21 y 30 horas,92693
Menos de 10 horas,87191


In [35]:
# Replacement mappings based on Estrato
# Replace 'Si' and 'No' with True and False
data_train_copy['FAMI_TIENEINTERNET'] = data_train_copy['FAMI_TIENEINTERNET'].replace({'Si': True, 'No': False})

# Create a custom function to handle NaN values
def replace_nan_with_boolean(row):
    if pd.notnull(row['FAMI_TIENEINTERNET']):
        return row['FAMI_TIENEINTERNET']
    else:
        estrato = row['FAMI_ESTRATOVIVIENDA']
        if estrato in {'Estrato 6', 'Estrato 4', 'Estrato 5'}:
            return True
        elif estrato in {'Estrato 1', 'Estrato 2', 'Estrato 3'}:
            return False
        else:
            return row['FAMI_TIENEINTERNET']  # Return the same if estrato does not match

# Apply the custom function to replace NaN values in the DataFrame
data_train_copy['FAMI_TIENEINTERNET'] = data_train_copy.apply(replace_nan_with_boolean, axis=1)

# Display the updated column values to verify changes
data_train_copy['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
True,604736
False,87764


In [36]:
# Original DataFrame column
data_train['FAMI_TIENEINTERNET'].value_counts()

,count
FAMI_TIENEINTERNET,
Si,592514
No,73357


In [37]:
# Replace 'Si' and 'No' with Boolean True and False
data_train_copy['ESTU_PAGOMATRICULAPROPIO'] = data_train_copy['ESTU_PAGOMATRICULAPROPIO'].replace({'Si': True, 'No': False})

# Replace NaN values with a random choice between True and False
data_train_copy['ESTU_PAGOMATRICULAPROPIO'] = data_train_copy['ESTU_PAGOMATRICULAPROPIO'].apply(lambda x: np.random.choice([True, False]) if pd.isna(x) else x)

# Display the updated column values to verify changes
data_train_copy['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
False,385521
True,306979


In [38]:
# Original DataFrame column
data_train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,382201
Si,303801


In [39]:
# Replacement mappings Estrato categories for int values
data_train_copy['FAMI_ESTRATOVIVIENDA'] = data_train_copy['FAMI_ESTRATOVIVIENDA'].replace({
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6
})

# Display the updated column values to verify changes
data_train_copy['FAMI_ESTRATOVIVIENDA'].value_counts()

<ipython-input-39-6b53a0655abf>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_train_copy['FAMI_ESTRATOVIVIENDA'] = data_train_copy['FAMI_ESTRATOVIVIENDA'].replace({


,count
FAMI_ESTRATOVIVIENDA,
2,237846
3,220594
1,117071
4,73373
5,27230
6,16386


In [40]:
# Display the updated column values to verify changes
data_train['FAMI_ESTRATOVIVIENDA'].value_counts()

,count
FAMI_ESTRATOVIVIENDA,
Estrato 2,232671
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [41]:
# Check that now no column has NaN values
for col in data_train_copy.columns:
    print(col, data_train_copy[col].isnull().any())

PERIODO False
ESTU_PRGM_ACADEMICO False
ESTU_PRGM_DEPARTAMENTO False
ESTU_VALORMATRICULAUNIVERSIDAD False
ESTU_HORASSEMANATRABAJA False
FAMI_ESTRATOVIVIENDA False
FAMI_TIENEINTERNET False
FAMI_EDUCACIONPADRE False
FAMI_EDUCACIONMADRE False
ESTU_PAGOMATRICULAPROPIO False
RENDIMIENTO_GLOBAL False


In [42]:
data_train_copy.head(5)

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
ID,,,,,,,,,,,
904256,20212,ENFERMERIA,BOGOTÁ,6.25,5.0,3,True,Técnica o tecnológica incompleta,Postgrado,False,medio-alto
645256,20212,DERECHO,ATLANTICO,3.25,0.0,3,False,Técnica o tecnológica completa,Técnica o tecnológica incompleta,False,bajo
308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,3.25,35.0,3,True,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,False,bajo
470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,4.75,0.0,4,True,No sabe,Secundaria (Bachillerato) completa,False,alto
989032,20212,PSICOLOGIA,ANTIOQUIA,3.25,25.0,3,True,Primaria completa,Primaria completa,False,medio-bajo
